In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
sys.path.append("..")
import numpy as np

CLEAN_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0/"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"


ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis'

DEVICE='cuda:1'
########## initulize reference data and target data
from alignment.data_preprocess import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH)
tar_datainit = DataInit(CLEAN_PATH,CLEAN_PATH,TAR_EPOCH)
ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


from alignment.ReferenceGenerator import ReferenceGenerator
gen = ReferenceGenerator(ref_provider=ref_provider, tar_provider=tar_provider,REF_EPOCH=REF_EPOCH,TAR_EPOCH=TAR_EPOCH,ref_model=ref_model,tar_model=tar_model,DEVICE=DEVICE)

absolute_alignment_indicates,predict_label_diff_indicates,predict_confidence_Diff_indicates,high_distance_indicates = gen.subsetClassify(18,0.8,0.3,0.05)


from representationTrans.trans_visualizer_border import visualizer
from singleVis.SingleVisualizationModel import VisModel
from singleVis.projector import TimeVisProjector
model = VisModel(ENCODER_DIMS, DECODER_DIMS)

I = np.eye(512)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")
vis = visualizer(ref_provider, I,I, np.dot(ref_provider.train_representation(TAR_EPOCH),I), projector, 200,[0,1],'tab10')


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 521.40it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 8458.58it/s]


absolute alignment indicates number: 106 label diff indicates number: 12 confidence diff indicates number: 16 high distance number: 97


In [7]:
import torch
import torch.nn.functional as F

# 假设X和Y是高维数据，每一行代表一个样本
X = torch.Tensor(ref_train_data)
Y = torch.Tensor(tar_train_data)

# 计算每行样本之间的皮尔逊相关系数
corr = F.cosine_similarity(X, Y, dim=1)

# 打印前10个样本的相关系数
print(corr[:10])


tensor([0.5297, 0.6251, 0.5927, 0.5306, 0.5791, 0.5667, 0.5847, 0.5635, 0.5695,
        0.5763])


In [10]:
X = tar_train_data
Y = ref_train_data
# 计算X和Y的均值和标准差
X_mean, X_std = np.mean(X, axis=0), np.std(X, axis=0)
Y_mean, Y_std = np.mean(Y, axis=0), np.std(Y, axis=0)

# 将X缩放到Y的尺度
X_scaled = (X - X_mean) * (Y_std / X_std) + Y_mean


In [9]:
import torch
import torch.nn.functional as F

# 假设X和Y是高维数据，每一行代表一个样本
X = torch.Tensor(X_scaled)
Y = torch.Tensor(Y)

# 计算每行样本之间的皮尔逊相关系数
corr = F.cosine_similarity(X, Y, dim=1)

# 打印前10个样本的相关系数
print(corr[:10])

tensor([0.5511, 0.6504, 0.6205, 0.5520, 0.6021, 0.5994, 0.6097, 0.5858, 0.5937,
        0.5988])


In [36]:
from CKA_utils.CKA import CKA, CudaCKA
np_cka = CKA()
# indicates = np.random.choice(aligned, size=10, replace=False)
indicates = np.random.choice(np.arange(5000), size=1000, replace=False)
print('CKA between absolute alignmnet subset is:',np_cka.kernel_CKA(X[indicates],Y[indicates]))

CKA between absolute alignmnet subset is: 0.8234975187095028


In [35]:
from CKA_utils.CKA import CKA, CudaCKA
np_cka = CKA()
# indicates = np.random.choice(aligned, size=10, replace=False)
indicates = np.random.choice(np.arange(5000), size=1000, replace=False)
print('CKA between absolute alignmnet subset is:',np_cka.kernel_CKA(X_scaled[indicates],Y[indicates]))

CKA between absolute alignmnet subset is: 0.8269703859772014


In [37]:
from AlignVisAutoEncoder.autoencoder import SimpleAutoencoder
from AlignVisAutoEncoder.data import DataLoaderInit
input_dim = 512
output_dim = 512

autoencoder = SimpleAutoencoder(input_dim,output_dim)

In [38]:
checkpoint = torch.load("/home/yifan/projects/deepdebugertool/DLVisDebugger/AlignVisAutoEncoder/checkpoints/dropout_no_f_epoch_200.pth")
autoencoder.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [39]:
# Transform Y into X's space
encoded_Y = autoencoder.encoder(torch.Tensor(Y))
encoded_Y = encoded_Y.detach().numpy()

In [50]:

from CKA_utils.CKA import CKA, CudaCKA
np_cka = CKA()
# indicates = np.random.choice(aligned, size=10, replace=False)
indicates = np.random.choice(np.arange(5000), size=1000, replace=False)
print('CKA between absolute alignmnet subset is:',np_cka.kernel_CKA(X[indicates],encoded_Y[indicates]))

CKA between absolute alignmnet subset is: 0.5663528197091249


In [63]:

from CKA_utils.CKA import CKA, CudaCKA
np_cka = CKA()
# indicates = np.random.choice(aligned, size=10, replace=False)
indicates = np.random.choice(np.arange(5000), size=1000, replace=False)
print('CKA between absolute alignmnet subset is:',np_cka.kernel_CKA(Y[indicates],encoded_Y[indicates]))

CKA between absolute alignmnet subset is: 0.7490385480257218
